### 错误问题纠正

In [1]:
# 导入数据
include("../src/readfiles.jl")

导入成功，内容概要：
    | 变量名 | 说明 |
    | ---- | ---- |
    | zh_triples/en_triples | 三元组 |
    | zh_subs/en_subs | 头实体 |
    | zh_objs/en_objs | 尾实体 |
    | zh_rels/en_rels | 关系 |
    | zh_entity/en_entity | 实体（头和尾） |
    | ILLs | 英文 => 中文对齐 |
    | ILLs_zh_en | 中文 => 英文对齐|
    | train_data | 训练集 |


In [2]:
# 处理错误问题的正则表达式
# 只纠正错误问题，而不会影响其他问题
lg, rg, substr = "([A-Z])", raw"([\w ,'()/–\-\.]+)", s"\g<1>\g<2>"
ill_rules(rep) = Regex(lg * rep * rg) => SubstitutionString(substr * rep)
ill_pattern = ill_rules.([
        "'s name", # 211
        "'s southern region", # 190
        " known", # 166
        "'s famous works", # 140
        "-related products", # 101
        "'s related products", # 63
        " related events", # 35
]);
push!(ill_pattern,
    r"([^A-Za-z0-9])-held parliament([^A-Za-z0-9]+)(?<![ ?])" => s"\g<1>\g<2>-held parliament", # 21
    r"which ([^A-Za-z0-9]) belongs([^A-Za-z0-9]+)(?<![ ?])" => s"\g<1>\g<2> belongs", # 16
);

In [3]:
ill_ques = String[] # 不能直接提取头实体的问题
valid_ques = String[] # 能直接提取头实体的问题
a1, a2 = 0, 0
for (que, sol) in train_data
    ob = replace(first(sol)[2], '_' => ' ')
    que = replace(que, ill_pattern...)
    occursin(ob, que) ? push!(valid_ques, que) : push!(ill_ques, que)
end
ill_ques

15-element Vector{String}:
 "Savez-vous ce qu’est la dynastie ?"
 "Savez-vous quels sont les pays participants"
 "who married the person of T-related eventhose Were the Days (song)?"
 "what is the position of the country related to B-related eventsyzantine Crete?"
 "which title does M-related event related personsecklenburg-Güstrow own?"
 "who is the key people of 2's data source016–17 AFC Wimbledon season?"
 "what is children of the person of T related eventhose Were the Days (song)?"
 "Saviez-vous que la série du produits liés à l'ENTITÉest?"
 "what is originated from the person of A related eventncient Corinth?"
 "where is the birthplace of A-related event related personsncient Corinth?"
 "who is the owner of the sources of 2 related data016–17 AFC Wimbledon season?"
 "what is the club that 2's head coach006 Santos FC season belongs to?"
 "Savez-vous qu'avant produits liés à l'ENTITÉest?"
 "which country is 2's season006 Santos FC season born?"
 "produits liés à l'ENTITÉbasé sur ce"

In [5]:
# 保存结果
open("../data/extract/valid_ques.txt", "w") do io
    for que in valid_ques
        println(io, que)
    end
end

### 调试过程

In [241]:
newspecial = String[]
for que in ill_ques
    ob = replace(first(train_data[que])[2], '_' => ' ')
    new_que = replace(que, ill_pattern...)
    if !occursin(ob, new_que)
        push!(newspecial, new_que)
        new_que != que && println(que,'\n', new_que)
    end
end
println(length(newspecial))

15


In [243]:
# que = newspecial[5]
# print(que)
# train_data[que]
println.(newspecial);

Savez-vous ce qu’est la dynastie ?
Savez-vous quels sont les pays participants
who married the person of T-related eventhose Were the Days (song)?
what is the position of the country related to B-related eventsyzantine Crete?
which title does M-related event related personsecklenburg-Güstrow own?
who is the key people of 2's data source016–17 AFC Wimbledon season?
what is children of the person of T related eventhose Were the Days (song)?
Saviez-vous que la série du produits liés à l'ENTITÉest?
what is originated from the person of A related eventncient Corinth?
where is the birthplace of A-related event related personsncient Corinth?
who is the owner of the sources of 2 related data016–17 AFC Wimbledon season?
what is the club that 2's head coach006 Santos FC season belongs to?
Savez-vous qu'avant produits liés à l'ENTITÉest?
which country is 2's season006 Santos FC season born?
produits liés à l'ENTITÉbasé sur ce
